In [1]:
import pandas as pd

# Cargar el archivo Excel 
df = pd.read_excel("../Data/datos_agronet.xlsx")

# Mapeo de nombres de mercado a categorías válidas
mercados_validos = {
    'Bogotá, D.C., Corabastos': 'Bogotá',
    'Corabastos': 'Bogotá',
    'Tunja': 'Tunja',
    'Tunja (Boyacá)': 'Tunja',
    'Duitama': 'Duitama',
    'Duitama (Boyacá)': 'Duitama',
    'Sogamoso (Boyacá)': 'Sogamoso'
}

# Aplicar mapeo
df['Mercado'] = df['Mercado'].map(mercados_validos)



In [2]:
df.Mercado.unique()

array(['Bogotá', 'Tunja', 'Sogamoso', 'Duitama'], dtype=object)

In [3]:
# Eliminar filas con mercados no deseados
df = df.dropna(subset=['Mercado'])

# Normalizar y convertir fechas
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') 
df['Fecha'] = df['Fecha'].str.replace('Viernes, ', '', regex=False)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%B %d de %Y', errors='coerce')

# Eliminar filas con fechas no convertidas correctamente
df = df.dropna(subset=['Fecha'])

# Eliminar duplicados
df = df.drop_duplicates()

# Verificar y convertir columna de precio
# Quitar caracteres no numéricos y convertir a float (por si hay errores de entrada)
df['Precio $/KG'] = pd.to_numeric(df['Precio $/KG'], errors='coerce')

# Eliminar filas con precios nulos o inconsistentes
df = df.dropna(subset=['Precio $/KG'])

# Resetear índice y mostrar dataset limpio
df = df.reset_index(drop=True)
print(df)

# Guardar el dataframe limpio (opcional)
df.to_excel("../Data/datos_agronet_limpio.xlsx", index=False)  


        Mercado             Producto      Fecha  Precio $/KG
0        Bogotá  Aguacate papelillo  2005-07-15      2200.00
1        Bogotá  Aguacate papelillo  2005-07-22      2340.00
2        Bogotá  Aguacate papelillo  2005-07-29      2355.00
3        Bogotá  Aguacate papelillo  2005-08-05      2410.00
4        Bogotá  Aguacate papelillo  2005-08-12      2410.00
...         ...                  ...        ...          ...
65279     Tunja           Zanahoria  2011-12-16       616.67
65280   Duitama           Zanahoria  2011-12-23       850.00
65281     Tunja           Zanahoria  2011-12-23       745.00
65282    Bogotá           Zanahoria  2011-12-23       677.78
65283  Sogamoso           Zanahoria  2011-12-23       680.00

[65284 rows x 4 columns]


## Limpieza de datos para el dataset Precios Coomproriente

In [4]:
import pandas as pd
import re

# Cargar el archivo Excel
df_coomproriente = pd.read_excel("../data/Precios_Coomproriente_05_06_25.xlsx")

# Normalizar nombres de columnas
df_coomproriente.columns = [col.strip() for col in df_coomproriente.columns]

# Validar columnas necesarias
if 'Producto' not in df_coomproriente.columns or 'Precio' not in df_coomproriente.columns or 'Cantidad' not in df_coomproriente.columns:
    raise ValueError("Faltan columnas requeridas: 'Producto', 'Precio' o 'Cantidad'.")

# Reemplazar "$" y "." en precio y convertir a float
df_coomproriente['Precio'] = df_coomproriente['Precio'].replace('[\$\.,]', '', regex=True)
df_coomproriente['Precio'] = pd.to_numeric(df_coomproriente['Precio'], errors='coerce')

df_coomproriente.describe()


,Precio,Fecha
count,1422.000000,1440
mean,64920.323488,2025-05-01 23:54:00
min,1400.000000,2025-03-27 00:00:00
25%,18000.000000,2025-04-14 00:00:00
50%,50000.000000,2025-04-30 00:00:00
75%,90000.000000,2025-05-15 00:00:00
max,402000.000000,2025-06-05 00:00:00
std,59981.484022,NaN


In [5]:
# Cambiar "50" a "1" en la columna cantidad para Aguacate Papelillo
def ajustar_cantidad(cantidad, producto):
    if pd.isna(cantidad):
        return cantidad
    if "aguacate papelillo" in producto.lower():
        return re.sub(r'\b50\b', '1', cantidad)
    return cantidad

df_coomproriente['Cantidad'] = df_coomproriente.apply(lambda row: ajustar_cantidad(row['Cantidad'], row['Producto']), axis=1)

# Función para extraer el número de Kg
def extraer_kg(cantidad):
    if pd.isna(cantidad):
        return None
    match = re.search(r'(\d+(\.\d+)?)\s*(kg|kilogramo)', cantidad, re.IGNORECASE)
    return float(match.group(1)) if match else None

# Aplicar extracción y eliminar filas sin Kg
df_coomproriente['Kg'] = df_coomproriente['Cantidad'].apply(extraer_kg)
df_coomproriente = df_coomproriente[df_coomproriente['Kg'].notna()]

# Calcular precio por Kg
df_coomproriente['Precio_por_kg'] = df_coomproriente['Precio'] / df_coomproriente['Kg']
#Crear una columna llamada Mercado y asignar el valor "Sogamoso" en todo el DataFrame
df_coomproriente['Mercado'] = 'Sogamoso'
df_coomproriente.to_excel("../data/Precios_Coomproriente_limpio.xlsx", index=False)  
